In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.font_manager as fm
import seaborn as sns
import warnings
from scipy.special import boxcox1p, inv_boxcox1p

pd.options.display.float_format = '{:.5f}'.format
fontpath = 'C:/Users/TaeSoo/AppData/Local/Microsoft/Windows/Fonts/NanumGothic.ttf'
%matplotlib inline
warnings.filterwarnings(action='ignore')

plt.rcParams['figure.dpi'] = 140
plt.rcParams['font.family'] = 'NanumGothic'

In [2]:
train = pd.read_csv('../원본데이터/train.csv')
test = pd.read_csv('../원본데이터/test.csv')

In [3]:
train_df = train.copy()
test_df = test.copy()

In [4]:
dataset = pd.concat([train_df, test_df], axis = 0)

In [5]:
suburb_df = pd.DataFrame({'count' : dataset.groupby(['start_latitude', 'start_longitude', 'end_latitude', 'end_longitude']).size()}).reset_index()
suburb_df

,start_latitude,start_longitude,end_latitude,end_longitude,count
0,33.24343,126.42203,33.24488,126.41909,6221
1,33.24451,126.42707,33.24950,126.43079,4213
2,33.24460,126.56208,33.24487,126.56441,6594
3,33.24460,126.56208,33.24761,126.56194,5206
4,33.24487,126.56441,33.24460,126.56208,6640
...,...,...,...,...,...
899,33.55352,126.73971,33.55430,126.73838,5267
900,33.55430,126.73838,33.55352,126.73971,5298
901,33.55430,126.73838,33.55532,126.73668,5308
902,33.55532,126.73668,33.55430,126.73838,5306


In [6]:
suburb_df.to_csv('temp_suburb.csv')

In [7]:
from geopy.geocoders import Nominatim

In [8]:
def geocoding_reverse(lat_lng_str): 
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    address = geolocoder.reverse(lat_lng_str)

    return address

In [9]:
def searching_suburb(df, latitude, longitude) :
    real_suburb_Name = []
    for idx in range(len(df)) :
        real_suburb_Name.append(geocoding_reverse(str(df.loc[idx, latitude]) + ', ' + str(df.loc[idx, longitude])))
    return real_suburb_Name

In [10]:
def making_address_col (address) :
    country = []
    post_code = []
    state = []
    city = []
    suburb = []
    street_address = []

    for idx in range(len(address)) :
        temp_address_list = address[idx][0].split(', ')
        country.append(temp_address_list[-1])

        if temp_address_list[-2].isdigit() == True :
            post_code.append(temp_address_list[-2])
            state.append(temp_address_list[-3])
            if len(temp_address_list) == 4 :
                city.append(temp_address_list[-4])
                suburb.append(np.nan)
                street_address.append(np.nan)
            elif len(temp_address_list) == 5 :
                city.append(temp_address_list[-4])
                suburb.append(temp_address_list[-5])
                street_address.append(np.nan)                
            else :
                city.append(temp_address_list[-4])
                suburb.append(temp_address_list[-5])
                street_address.append(temp_address_list[-6])  

        else :
            post_code.append(np.nan)
            state.append(temp_address_list[-2])
            if len(temp_address_list) == 3 :
                city.append(temp_address_list[-3])
                suburb.append(np.nan)
                street_address.append(np.nan)
            elif len(temp_address_list) == 4 :
                city.append(temp_address_list[-3])
                suburb.append(temp_address_list[-4])
                street_address.append(np.nan)                
            else :
                city.append(temp_address_list[-3])
                suburb.append(temp_address_list[-4])
                street_address.append(temp_address_list[-5])  

    address_df = pd.DataFrame(data = list(zip(country, post_code, state, city, suburb, street_address)), columns = ['country', 'post_code', 'state', 'city', 'suburb', 'street_address'])
    return address_df

In [11]:
start_address = searching_suburb(suburb_df, 'start_latitude', 'start_longitude')
end_address = searching_suburb(suburb_df, 'end_latitude', 'end_longitude')

In [12]:
start_address_df = making_address_col(start_address)
end_address_df = making_address_col(end_address)

In [13]:
start_address_df.loc[[416, 417], 'suburb'] = '해안동'
start_address_df.loc[507, 'suburb'] = '연동'
start_address_df.loc[[i for i in range(535, 539)], 'suburb'] = '대천동'
start_address_df.loc[[196], 'suburb'] = '강정동'
start_address_df.loc[[298], 'suburb'] = '안덕면'
start_address_df.loc[[368], 'suburb'] = '표선면'
start_address_df.loc[[631], 'suburb'] = '월평동'
start_address_df.loc[[638,639, 640], 'suburb'] = '노형동'
start_address_df.loc[[645], 'suburb'] = '오라이동'
start_address_df.loc[[655], 'suburb'] = '조천읍'
start_address_df.loc[[670, 671, 680, 681], 'suburb'] = '애월읍'
start_address_df.loc[714, 'suburb'] = '외도일동'
start_address_df.loc[[798], 'suburb'] = '일도이동'
start_address_df.loc[[888], 'suburb'] = '구좌읍'

In [14]:
end_address_df.loc[[198], 'suburb'] = '강정동'
end_address_df.loc[[259], 'suburb'] = '안덕면'
end_address_df.loc[[367], 'suburb'] = '표선면'
end_address_df.loc[[416, 417], 'suburb'] = '해안동'
end_address_df.loc[[518, 533, 548, 554], 'suburb'] = '대천동'
end_address_df.loc[[492, 634, 636, 641], 'suburb'] = '노형동'
end_address_df.loc[[646], 'suburb'] = '오라이동'
end_address_df.loc[[654], 'suburb'] = '조천읍'
end_address_df.loc[[665, 667, 682], 'suburb'] = '애월읍'
end_address_df.loc[715, 'suburb'] = '외도일동'
end_address_df.loc[[814], 'suburb'] = '일도이동'
end_address_df.loc[[889], 'suburb'] = '구좌읍'

start_suburb  

196 강정동  
298 안덕면  
368 표선면  
631 월평동  
638, 639, 640 노형동  
645 오라이동  
655 조천읍  
670, 671 애월읍  
680, 681 애월읍  
714 외도일동  
798 일도이동  
888 구좌읍  
416, 417 해안동  
535 ~ 538 대천동  

end_suburb

198	대한민국	63566	제주특별자치도	서귀포시	강정동
259	대한민국	63522	제주특별자치도	서귀포시	안덕면
367	대한민국	63623	제주특별자치도	서귀포시	표선면
416	대한민국	NaN	제주특별자치도	제주시	해안동
417	대한민국	NaN	제주특별자치도	제주시	해안동
518	대한민국	NaN	제주특별자치도	제주시	대천동
533	대한민국	NaN	제주특별자치도	제주시	대천동
548	대한민국	NaN	제주특별자치도	제주시	대천동
554	대한민국	NaN	제주특별자치도	제주시	대천동
634	대한민국	63103	제주특별자치도	제주시	노형동
636	대한민국	63103	제주특별자치도	제주시	노형동
641	대한민국	63103	제주특별자치도	제주시	노형동
646	대한민국	63147	제주특별자치도	제주시	오라이동
654	대한민국	63343	제주특별자치도	제주시	조천읍
665	대한민국	63051	제주특별자치도	제주시	애월읍
667	대한민국	63058	제주특별자치도	제주시	애월읍
682	대한민국	63051	제주특별자치도	제주시	애월읍
715	대한민국	63070	제주특별자치도	제주시	외도일동
814	대한민국	63260	제주특별자치도	제주시	일도이동
889	대한민국	63359	제주특별자치도	제주시	구좌읍

In [15]:
start_address_df

,country,post_code,state,city,suburb,street_address
0,대한민국,63545,제주특별자치도,서귀포시,중문관광로,NaN
1,대한민국,63545,제주특별자치도,서귀포시,천제연로178번길,NaN
2,대한민국,63595,제주특별자치도,서귀포시,서귀동,태평로
3,대한민국,63595,제주특별자치도,서귀포시,서귀동,태평로
4,대한민국,63595,제주특별자치도,서귀포시,서귀동,이중섭로
...,...,...,...,...,...,...
899,대한민국,63357,제주특별자치도,제주시,김녕리,일주동로
900,대한민국,63357,제주특별자치도,제주시,김녕리,일주동로
901,대한민국,63357,제주특별자치도,제주시,김녕리,일주동로
902,대한민국,63357,제주특별자치도,제주시,동복리,구좌해안로


In [16]:
road_df = pd.concat([start_address_df['suburb'], end_address_df['suburb']], axis = 1)
road_df.columns = ['start_suburb', 'end_suburb']

In [17]:
geo_road_df = pd.concat([suburb_df[['start_latitude', 'start_longitude', 'end_latitude', 'end_longitude']], road_df[['start_suburb', 'end_suburb']]], axis = 1)

In [18]:
geo_road_df.to_csv('suburb_naming.csv', encoding = 'utf-8-sig')